# santander-value-prediction-challenge

# import

In [ ]:
import numpy as np
import pandas as pd
import tqdm
import os
from collections import deque
from collections import Counter
import codecs
import copy
from sklearn.metrics import confusion_matrix
import sys
from platform import python_version
from datetime import datetime
import time
import shutil
from sklearn.preprocessing import OneHotEncoder
import csv
import math

from sklearn.feature_selection import RFE



from sklearn.ensemble import VotingRegressor

#from sklearn.model_selection import train_test_split

#from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import roc_auc_score, f1_score
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

#from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate


from plotly import subplots

import plotly.graph_objs as go
import plotly.offline as py




# environment

In [ ]:
print(python_version())

In [ ]:
sys.getdefaultencoding()

In [ ]:
%autosave 0

In [ ]:
#最大表示列数の指定（ここでは5000列を指定）
pd.set_option('display.max_columns', 5000)

In [ ]:
#最大表示行数の指定（ここでは200行を指定）
pd.set_option('display.max_rows', 50)

# define

In [ ]:
# n_estimators:ここを増やすと精度は上がるが遅い。。。
ESTIMATOR_NUM=10
RANDOM_STATE_VAL = 0
target_col_name = "target"
id_col_name = "ID"
drop_columns =  []

#FEATURE_NUM = 2000
#FEATURE_NUM = 1500
#FEATURE_NUM = 1250
#FEATURE_NUM = 1000
#FEATURE_NUM = 950
#FEATURE_NUM = 900
#FEATURE_NUM = 800
#FEATURE_NUM = 500
#FEATURE_NUM = 100
#FEATURE_NUM = 90
#FEATURE_NUM = 50
FEATURE_NUM = 40

# 特徴量削減ステップ
SELECTOR_STEP = .05

CV_VAL = 5



# function

In [ ]:
def plotDistribution(rowcount_tuple, target_value, title = '') :
    plotObj = subplots.make_subplots(rows=1, cols=2, 
                                     subplot_titles=('Scatter', 'Histogram',),
                                    )
    plotObj.append_trace(go.Scatter(x=rowcount_tuple, y=np.sort(target_value), mode='lines', connectgaps=True,),
                            1,1,
                           )
    plotObj.append_trace(go.Histogram(x=target_value), 
                            1,2,
                           )
    plotObj['layout'].update(title = title, titlefont = dict(family = 'Arial', size = 36,), 
                                paper_bgcolor = '#d3b8d7', plot_bgcolor = '#71e1df',
                               )
    return py.iplot(plotObj)


In [ ]:
def dispalyDf(readDf, dispFlag, row_num, dispLabel):
    print("========================= {} =========================".format(dispLabel))
    row_num , col_num = np.shape(readDf)
    print("row_num = {}, col_num = {}".format(row_num, col_num))
    if dispFlag:
        display(readDf.head(row_num))
    print("++++++ None Count All++++++")
    print(readDf.isnull().values.sum())
    print("++++++ None Count ++++++")
    print(readDf.isnull().sum())
    print("++++++ Col Type ++++++")
    print(readDf.dtypes)
#     print("++++++ Unique By Col（Noneは含まないので注意） ++++++")
#     for col_name in readDf.columns.tolist():
#         print("{} : {}".format(col_name, readDf[col_name].nunique()))

In [ ]:
def chkVal(expect_label, expect_val, act_label, act_val):
    if expect_val != act_val:
        raise Exception("Val Error !! expect({}): {}, act({}) : {}"
                        .format(expect_label, expect_val, act_label, act_val))

In [ ]:
# test用のカラムのDataFrame
def getAdjustTestColDf(inputDf, train_col_list):
    not_enough_col_list = set(train_col_list) - set(inputDf.columns.tolist())
    print("# trainに有ってtestにないカラム名のリスト")
    print("{} : {}".format(len(not_enough_col_list), not_enough_col_list))
        
    dataLen = len(inputDf)
        
    retDf = inputDf.join(
        pd.DataFrame({col_name : [0]*dataLen  for col_name  in not_enough_col_list },
                     dtype=np.float, columns=not_enough_col_list)
    ).loc[:, train_col_list]
    
    # カラムリストが一致するかチェック
    if retDf.columns.tolist() != train_col_list:
        raise Exception("col_name_list not Match train and test.")

    # データ行数が一致するかチェック
    chkVal("dataLen", dataLen, "retDf", len(retDf))

    return retDf

In [ ]:
def getScalerDf(inputDf):
    row_num, col_num = np.shape(inputDf)
    
    # 平準化
    #scaler = StandardScaler()
    scaler = MinMaxScaler()
    
    val_array = scaler.fit_transform(np.array(inputDf.values.tolist()))
    standardScDf = pd.DataFrame(val_array, dtype=np.float, columns = inputDf.columns.tolist())
    
    sc_row_num, sc_col_num = np.shape(standardScDf)

    # 行数チェック
    chkVal("expect_row_num", row_num, "sc_row_num", sc_row_num)
    # 列数チェック
    chkVal("expect_col_num", col_num, "sc_col_num", sc_col_num)

    
    del val_array
    return standardScDf


In [ ]:
def getNplogDf(inputDf):
    return np.log1p(inputDf)

In [ ]:
def getDeconvNplogDf(inputData):
    return np.exp(inputData) - 1


# process

In [ ]:
act_train_read_Df = pd.read_csv('input/train.csv')
act_test_read_Df = pd.read_csv('input/test.csv')

In [ ]:
act_train_Df = act_train_read_Df.copy()
act_test_Df = act_test_read_Df.copy()
del act_train_read_Df, act_test_read_Df

In [ ]:
dispalyDf(act_train_Df, True, 10, "act_train_Df")

In [ ]:
dispalyDf(act_test_Df, True, 10, "act_test_Df")

In [ ]:
################################################

In [ ]:
# train:データのDataFrameと、ターゲットのデータフレームに分ける。
train_target_Ser = act_train_Df[target_col_name]
act_train_Df = act_train_Df.loc[:,[col_name for col_name in act_train_Df.columns.tolist() if col_name not in [id_col_name,target_col_name]]]
#print(set(train_target_Ser))

In [ ]:
# test:idとデータのDataFrameを抽出。
test_id_Df = act_test_Df.loc[:,[id_col_name]]
act_test_Df = act_test_Df.loc[:,[col_name for col_name in act_test_Df.columns.tolist() if col_name != id_col_name]]

In [ ]:
# targetの分布を調べる
# 
plotDistribution(tuple(range(act_train_Df.shape[0])), train_target_Ser, 'Target')

# logを取る
plotDistribution(tuple(range(act_train_Df.shape[0])), np.log1p(train_target_Ser), 'Target - log1p')


In [ ]:
# 値のユニーク数が１のカラムをdropリストに追加する
drop_columns.extend([col_name for col_name in act_train_Df.columns.tolist()
                    if act_train_Df[col_name].nunique() == 1 not in drop_columns])

In [ ]:
def convDescFeature(inputDf):
    retDf = inputDf.copy()
    
    retDf['quantile'] = \
        inputDf[inputDf > 0].quantile(0.5, axis=1)
    retDf['max'] = \
        inputDf.max(axis=1)
    retDf['skew'] = \
        inputDf.skew(axis=1)
    retDf['kurtosis'] = \
        inputDf.kurtosis(axis=1)
    retDf['sum_all'] = \
        inputDf.sum(axis=1)
    retDf['variance'] = \
        inputDf.var(axis=1)
    
    retDf = retDf.loc[:, ["quantile", "max", "skew", "kurtosis", "sum_all", "variance"]] 
    
    return retDf

In [ ]:
###### 前処理 ######
def getBeroreProcessDf(input_Df, drop_columns):
    retDf = input_Df.copy()
    retDf = retDf.drop(drop_columns, axis=1)
    
    columns = retDf.columns.tolist()
    
    
    for col_name in columns:
        if retDf[col_name].dtype == object:
            # 文字列のカラムの加工処理
            pass
#            retDf[col_name] = retDf[col_name].fillna('0')
#            retDf[col_name] = retDf[col_name].apply(lambda x: x.split(' ')[1])
#         # int64に変換する。
#         retDf[col_name] = pd.to_numeric(retDf[col_name])

    # 0→nan
    retDf = retDf.replace(0, np.nan)

#    retDf =  getScalerDf(retDf)
    retDf =  getNplogDf(retDf)

    #### desc特徴量に変換 #####
    retDf = convDescFeature(retDf)

    # 0←nan
    retDf = retDf.replace(np.nan, 0)

    return retDf
    

In [ ]:
### trainの前処理

In [ ]:
act_train_Df_2 = getBeroreProcessDf(act_train_Df, drop_columns)

In [ ]:
dispalyDf(act_train_Df_2, True, 10, "act_train_Df_2")

In [ ]:
### testの前処理

In [ ]:
act_test_Df_2 = getBeroreProcessDf(act_test_Df, drop_columns)

In [ ]:
dispalyDf(act_test_Df_2, True, 10, "act_test_Df_2")

In [ ]:
np.log1p(train_target_Ser)

In [ ]:
####################################
##### 次元削減(特徴量選択) を使用する
##### ↓特徴量選択(Ridge)
####################################
def getSelectorFeature(inputFeatureDf, model, y_Df,feature_num, selector_step):
    # 特徴量因子の重要度を推定する分類器をRidgeに設定
    # 1回のstepで削除する次元数は5%ずつとする
    
    print("特徴量選択数 = ",feature_num)
    if(feature_num < 0):
        # デフォルト値
        print("n_features_to_select = None に設定")
        feature_num = None
    elif feature_num > len(inputFeatureDf.columns.tolist()):
        print ("feature_num({}) is Over colnum({})".format(feature_num, len(inputFeatureDf.columns.tolist())))
        # 次元削減せずに返却する
        return inputFeatureDf
    
    # 特徴量選択
    # step:特徴量削除の速度。一度の再帰処理により指定ステップ分の特徴量が消滅する。
#    selector = RFE(estimator=Ridge(random_state=RANDOM_STATE_VAL, alpha=1.3),
    selector = RFE(estimator=model,
                   n_features_to_select=feature_num,
                   step=selector_step)
    selector.fit(inputFeatureDf,y_Df.values.ravel())
    #selector.fit(inputFeatureDf, y_Df.iloc[:,0])

    # 削除対象配列(Falseが削除対象)
    selFlagArray = selector.support_
    
    # 現状のカラム名を取得
    columnIndex = inputFeatureDf.columns
    
    # 削除対象カラム名リスト
    drop_column_list = list()
    # 削除対象配列 でループ(i=0,1,2,・・・)
    print(len(selFlagArray))
    for i in range(len(selFlagArray)):
        # Falseの場合に、削除対象リストに追加
        if(selFlagArray[i] == False):
            drop_column_list.append(columnIndex[i])

    # 不要な列削除
    #print("削除対象カラム = " , drop_column_list)
    inputFeatureNewDf = inputFeatureDf.drop(drop_column_list, axis=1)
    

    return inputFeatureNewDf


In [ ]:
####################################
##### 次元削減(nuniqueによる特徴量選択) を使用する
####################################
def getSelectorFeatureByNunique(inputFeatureDf, feature_num):
    temp_dict = {
        "col_name":[],
        "nunique":[]
    }
    for col_name in inputFeatureDf.columns.tolist():
        temp_dict["col_name"].append(col_name)
        temp_dict["nunique"].append(inputFeatureDf[col_name].nunique())

    rankDf = pd.DataFrame(temp_dict)

    rankDf['順位'] = (rankDf.rank(method='min',ascending=False)["nunique"]).astype('int')
    rankDf = rankDf.sort_values('順位', ascending=True)
    rankDf = rankDf[rankDf["順位"] <= feature_num]
    
    tempDf = inputFeatureDf.loc[:, rankDf["col_name"].values.tolist()]
    
    return tempDf

In [ ]:
# モデル入手
def getModel(train_Df, y_train, random_state_val, cv_val, feature_num, selector_step):
    print("cv_val = ",cv_val)
    model_def_list = [
        [
            "Ridge",
             #Ridge(random_state=random_state_val, alpha=100)
             #Ridge(random_state=random_state_val, alpha=20)
            Ridge(random_state=random_state_val, alpha=10)
             #Ridge(random_state=random_state_val, alpha=100,tol=100000)
            #Ridge(random_state=random_state_val, alpha=1.3)
            #Ridge(random_state=random_state_val)
        ],
        [
            "OLS",
            LinearRegression()
        ],
        [
            "SGDRegressor",
#            SGDRegressor(random_state=random_state_val, alpha=0.3)
            SGDRegressor(random_state=random_state_val)
        ]
    
    ]
    
    
    ret_model_list = []
    test_score_list = []
    
    
    for model_elem in model_def_list:
        print(" ==================== {} ====================".format(model_elem[0]))
        model = model_elem[1]
        
        start = time.time()
        # 特徴量選択
#        train_Df_new = getSelectorFeature(train_Df, model, y_train, feature_num, selector_step)
#        train_Df_new = getSelectorFeatureByNunique(train_Df, feature_num)
        train_Df_new = train_Df

        
        model.fit(train_Df_new, y_train.values.ravel())
        ret_model_list.append(model)

        cv_results = cross_validate(model,
                                                         train_Df_new,
                                                         y_train.values.ravel(),      #1次元のARRAY
                                                         cv=cv_val,
                                                         scoring='r2')
#         print(cv_results)
#         print(cv_results["test_score"])
        print("test_score = ", np.mean(cv_results["test_score"]))
        test_score_list.append(np.mean(cv_results["test_score"]))

        elapsed_time = time.time() - start
        print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")
    
    print("----------------------------------------------------------------------------------------")
    # test_score が最大のアルゴリズムを適用する
    print(test_score_list)
    selected_index = np.argmax(test_score_list)
    print("selected_model : {}".format(model_def_list[selected_index][0]))
    return ret_model_list[selected_index],  train_Df_new.columns.tolist()


In [ ]:
# モデル入手
# model, train_col_name_list = getModel(act_train_Df_2, train_target_Ser, RANDOM_STATE_VAL, CV_VAL,
#                                       FEATURE_NUM, SELECTOR_STEP)
model, train_col_name_list = getModel(act_train_Df_2, np.log1p(train_target_Ser), RANDOM_STATE_VAL, CV_VAL,
                                     FEATURE_NUM, SELECTOR_STEP)


In [ ]:
act_test_Df_3 = getAdjustTestColDf(act_test_Df_2, train_col_name_list)

In [ ]:
# train-test間でカラムが一致しているかチェック
chkVal("train_col_list", train_col_name_list, "test_col_list", act_test_Df_3.columns.tolist())

In [ ]:
dispalyDf(act_test_Df_3, True, 10, "act_test_Df_3")

In [ ]:
model

In [ ]:
#### 予測 ####
test_predict = model.predict(act_test_Df_3)
target_Df = pd.DataFrame(
    {
        target_col_name: test_predict 
    })

In [ ]:
target_Df

In [ ]:
getDeconvNplogDf(target_Df)

In [ ]:
output_Df = test_id_Df.join(getDeconvNplogDf(target_Df))

In [ ]:
output_Df[output_Df[target_col_name] < 0]

In [ ]:
dispalyDf(output_Df, True, 10, "output_Df")

In [ ]:
# ディレクトリなければ作る
output_dir_path = os.path.join(os.getcwd(), 'output')
print(output_dir_path)
if not os.path.exists(output_dir_path):
    os.makedirs(output_dir_path) 

In [ ]:
# 出力
output_Df.to_csv('output/submission.csv',
                 sep=",",
                 header=True,
                 index=False,
                 mode="w",
                 encoding="utf-8",
                 line_terminator="\n"
)

In [ ]:
########################################################

In [ ]:
raise Exception("END♪")

In [ ]:
########################################################